In [ ]:
# Task 2.1
class Node:
    def __init__(self, data_value = -1, index = -1, left_node = -1, right_node=-1):
        self._data_value = data_value
        self._index = index
        self._left_node = left_node
        self._right_node = right_node



    def __repr__(self):
        return f"[{self._index}]{self._left_node}:{self._data_value}:{self._right_node}"
    ## skip the getters/setters



In [2]:
# Task 2.2
def calculate_nodes(level:int):
    if level < 0:
        return None
    ret = (2**level) - 1
    #print(f"A {level} level number tree has {ret} nodes")
    return ret

In [3]:
#Task 2.3
calculate_nodes(1)
calculate_nodes(3)
calculate_nodes(5)
calculate_nodes(10)

1023

In [ ]:
#Task 2.4
def create_empty_tree(list_nodes, level):
    if level < 0:
        return None
    list_nodes = [ Node(index=i) for i in range(calculate_nodes(level))]
    return list_nodes

In [ ]:
def num_leaf_nodes(level):
    return 2**(level-1)

#Task 2.5
def assign_leaf_nodes(list_nodes, level):
    for n, node in enumerate( list_nodes[:num_leaf_nodes(level)], 1):
        node._data_value = n
    return list_nodes

In [ ]:
# Task 2.6
## moving window
def create_number_tree(list_nodes, level):
    lb = 0
    for l in range(level,0,-1):
        ub = lb + num_leaf_nodes(l)-1
        stack = list_nodes[lb:ub+1]
        offset = 1
        while len(stack) > 1:
            left_node =  stack.pop(0)
            right_node = stack.pop(0)
            list_nodes[ub + offset]._data_value = left_node._data_value + right_node._data_value
            list_nodes[ub + offset]._left_node = list_nodes.index(left_node)
            list_nodes[ub + offset]._right_node = list_nodes.index(right_node)
            offset +=1
        lb = ub+1

In [ ]:
level = 3
list_nodes = create_empty_tree([],level)
list_nodes = assign_leaf_nodes(list_nodes,level)
create_number_tree(list_nodes,level)
print(list_nodes)

In [ ]:
## Queue technique
def create_number_tree2(list_nodes, level):
    ## put the leaf nodes in a queue
    pending_queue = [ i for i in range(num_leaf_nodes(level))]
    free_index = pending_queue[-1] + 1

    ## progressive create the next level
    while len(pending_queue) > 1:
        left = pending_queue.pop(0)
        right = pending_queue.pop(0)
        list_nodes[free_index]._left_node = left
        list_nodes[free_index]._right_node = right
        list_nodes[free_index]._data_value = list_nodes[left]._data_value + list_nodes[right]._data_value
        pending_queue.append(free_index)
        free_index +=1


In [ ]:
## Merge sort technique
## requires a index to be be stored in the node
def create_number_tree3(list_nodes, level):
    def merge_nodes(left_node, right_node, next_index):
        return Node( left_node._data_value + right_node._data_value,
                    next_index,
                    left_node._index,
                    right_node._index
                        )
    leaf_layer= list_nodes[:num_leaf_nodes(level)]
    next_index = len(leaf_layer)
    while len(leaf_layer) > 1:
        next_layer = []
        while len(leaf_layer) > 1:
            new_node = merge_nodes(leaf_layer.pop(0), leaf_layer.pop(0), next_index)
            list_nodes[next_index] = new_node
            next_layer.append( new_node)
            next_index +=1
        leaf_layer = next_layer


In [ ]:
level = 7
list_nodes = create_empty_tree([],level)
list_nodes = assign_leaf_nodes(list_nodes,level)
create_number_tree3(list_nodes,level)
#for i in range(len(list_nodes)):
#    print(f"[{i}]{list_nodes[i]}")



In [ ]:
## print nodes at tree levels
ub =  calculate_nodes(level)-1

for l in range(1,level+1):
    n = num_leaf_nodes(l)
    lb = ub-n + 1
    for i in range(lb, ub+1):
        print(f"{list_nodes[i]}", end=",")
    ub = lb-1
    print()


____
#### Recursive Solution
This solution requires the root node to be at index 0
and the leaf nodes to be at the end of the list

In [ ]:
## root node is at index 0
def create_number_tree_reverse(list_nodes, level):
    def calc_node(node_index):
        left_index = node_index * 2 + 1
        right_index = node_index * 2 + 2
        if list_nodes[left_index]._data_value == -1: # not leaf
            calc_node(left_index)
        left_value = list_nodes[left_index]._data_value
        if list_nodes[right_index]._data_value == -1:
            calc_node(right_index)
        right_value = list_nodes[right_index]._data_value

        list_nodes[node_index]._data_value = left_value + right_value
        list_nodes[node_index]._left_node = left_index
        list_nodes[node_index]._right_node = right_index
    calc_node(0)

In [ ]:
create_number_tree_reverse(list_nodes, level)

___
##### Solution that requires a reverse map of the indices in the array

In [ ]:
## Helper ADT to help with the tree positions and array indices
class Helper:
    def __init__(self, level):
        num_nodes = calculate_nodes(level)
        indexes = [ i for i in range(num_nodes-1,-1,-1)]
        self.__reverse_indexes = []
        lb=0
        for n in [ num_leaf_nodes(level) for level in range(1, level+1) ]:
            ub = lb + n
            self.__reverse_indexes += list(reversed(indexes[lb:ub]))
            lb = ub
    def get_index(self, i):
        return self.__reverse_indexes[i]

    def get_position(self, i):
        return self.__reverse_indexes.index(i)

In [ ]:
## root node is at index n-1
def create_number_tree4(list_nodes, level):
    def calc_node(node_index):
        node_position = look_up.get_position(node_index)
        left_index = look_up.get_index(node_position * 2 + 1)
        right_index = look_up.get_index(node_position * 2 + 2)

        if list_nodes[left_index]._data_value == -1:
            calc_node(left_index)
        left_value = list_nodes[left_index]._data_value
        if list_nodes[right_index]._data_value == -1:
            calc_node(right_index)
        right_value = list_nodes[right_index]._data_value
        list_nodes[node_index]._data_value = left_value + right_value
        list_nodes[node_index]._left_node = left_index
        list_nodes[node_index]._right_node = right_index
    look_up = Helper(level)
    calc_node(len(list_nodes)-1) ## root is at the end

In [ ]:
level = 3
list_nodes = create_empty_tree([],level)
list_nodes = assign_leaf_nodes(list_nodes,level)
create_number_tree4(list_nodes, level)

In [ ]:
## print nodes at tree levels
ub =  calculate_nodes(level)-1

for l in range(1,level+1):
    n = num_leaf_nodes(l)
    lb = ub-n + 1
    for i in range(lb, ub+1):
        print(f"{list_nodes[i]}", end=",")
    ub = lb-1
    print()


____
#### Node based

In [ ]:
# Task 2.1
class Node:
    def __init__(self, data_value = -1, left_node = None, right_node=None):
        self._data_value = data_value
        self._left_node = left_node
        self._right_node = right_node

    def get_data(self):
        return self._data_value
    def get_left(self):
        return self._left_node
    def get_right(self):
        return self._right_node


    def bfs(self):
        ret = []
        def _recur_bfs(next_level):
            if len(next_level) > 0 :
                node = next_level.pop(0)
                ret.append(node._data_value)
                if node._left_node:
                    next_level.append(node._left_node)
                if node._right_node:
                    next_level.append(node._right_node)
                _recur_bfs(next_level)

        _recur_bfs([self])
        return ret


    def __repr__(self):
        return f"{self._data_value}"
    ## skip the getters/setters

# number of nodes in level, level 1 is root node
def num_leaf_nodes(level):
    return 2**(level-1)

## Total nodes in a tree with n levels

def calculate_nodes(n:int):
    ret = 0
    for level in range(1,n+1):
        ret += num_leaf_nodes(level)
    return ret



In [ ]:
# merge nodes into a node by creating a tree with 3 nodes
def merge_nodes(left: Node, right: Node):
    return Node(
        left._data_value + right._data_value,
        left,
        right
    )
# merge list in ascending order of element's values in list
def merge_sorted_L(left: list, right: list):
    ret = []
    while left and right:
        if left[0] < right[0]:
            ret.append(left.pop(0))
        else:
            ret.append(right.pop(0))
    return ret +  left + right

## a generic algorithm to merge a List of list into a single List
def merge(L:list, F):

    if len(L) == 1:
        return L[0]
    else:
        L.append(
            F(
                L.pop(0),
                L.pop(0)
            )
        )
        return merge(L, F)

In [ ]:
## a leaf node is a single tree with a root node
def create_leaf_nodes(level):
    return  [Node(i)  for i in range(1,num_leaf_nodes(level)+1) ]


In [ ]:
level = 2
trees = create_leaf_nodes(level)
tree = merge(trees, merge_nodes)

In [ ]:
from TreeUtils2 import print_tree
print_tree(tree)


In [ ]:
import random
L = [ [random.randint(1,99)] for _ in range(20)]
print(L)
merge(L, merge_sorted_L)

____
##### Node based Array based hybrid
    - array stores the data
    - Node stores the index of the data and the subtree references

In [ ]:

class Node:
    ## does not contain the data, only contains relationships between nodes
    def __init__(self, index = -1, left_node = None, right_node = None):
        self._index = index
        self._left_node = left_node
        self._right_node = right_node

    def insert(self, new_index, data, cache): ## BST insert
        if data < cache[self._index]:
            if self._left_node == None:
                self._left_node = Node(index = new_index)
            else:
                self._left_node.insert(new_index, data, cache)
        else:
            if self._right_node == None:
                self._right_node = Node(index = new_index)
            else:
                self._right_node.insert(new_index, data, cache)

    def pre_order(self, cache):

        print(cache[self._index])
        if self._left_node:
            self._left_node.pre_order(cache)
        if self._right_node:
            self._right_node.pre_order(cache)

    def in_order(self, cache):

        if self._left_node:
            self._left_node.in_order(cache)
        print(cache[self._index])
        if self._right_node:
            self._right_node.in_order(cache)

    def post_order(self, cache):

        if self._left_node:
            self._left_node.post_order(cache)
        if self._right_node:
            self._right_node.post_order(cache)
        print(cache[self._index])

    def level_order(self, cache, queue):
        if queue:
            node = queue.pop(0)
            print(cache[node._index])
            if node._left_node:
                queue.append(node._left_node)
            if node._right_node:
                queue.append(node._right_node)
            node.level_order(cache, queue)




    def __repr__(self):
        return f"<[{self._index}]{self._left_node}:{self._right_node}>"
    ## skip the getters/setters


In [ ]:
def calculate_nodes(level:int):
    if level < 0:
        return None
    ret = (2**level) - 1
    #print(f"A {level} level number tree has {ret} nodes")
    return ret

In [ ]:
class Tree:
    def __init__(self, level):
        self._cache = [ None for i in range(calculate_nodes(level))]
        self.free = 0
        self.root = None
    def is_full(self):
        return self.free == len(self._cache)

    def insert_order(self, data):
        if self.is_full(): # full
            return
        self._cache[self.free] = data
        if self.root == None:
            self.root = Node(index = self.free)
        else:
            self.root.insert(self.free, data, self._cache)
        self.free+=1
    def pre_order(self):
        if self.root:
            self.root.pre_order(self._cache)
    def in_order(self):
        if self.root:
            self.root.in_order(self._cache)
    def post_order(self):
        if self.root:
            self.root.post_order(self._cache)
    def level_order(self):
        if self.root:
            self.root.level_order(self._cache,[self.root])


    def num_leaf_nodes(level):
            return 2**(level-1)

    def assign_leaf_nodes(self, level):
        n = Tree.num_leaf_nodes(level)
        roots = []
        for i in range(n):
            self._cache[i] = i+1
            roots.append(Node(index=i))
        self.free = n
        return roots

    def merge_nodes(self, left: Node, right: Node):
        if self.is_full():
            return
        self._cache[self.free] = self._cache[left._index] + self._cache[right._index]
        new_node = Node(self.free,left,right)
        self.free+=1
        return new_node



In [ ]:
level=3
tree = Tree(level)
roots = tree.assign_leaf_nodes(level)
## build tree
while len(roots) > 1:
    left = roots.pop(0)
    right = roots.pop(0)
    roots.append( tree.merge_nodes(left, right))
tree.root =  roots[0]



In [ ]:
tree.level_order()
